# Shut the Box

## Step 0: Utils for Jupyter Notebooks

### Add button to Show/Hide code (to focus on outputs)

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show Code"></form>''')

### Add magic command to store python variables in csv file

In [6]:
# credit to sirex, https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Step 1: Generate models

In [7]:
# Edit this cell to define parameters for your model...

boards = 12 # Number of boards to use
sides = 6 # Number of sides per die
die = 2 # Number of die to roll each round

max_score = boards * (boards+1) // 2

In [8]:
%%writetemplate ShutTheBox/params/stb_params_high.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 1

In [9]:
%%writetemplate ShutTheBox/params/stb_params_low.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 2

In [10]:
%%writetemplate ShutTheBox/params/stb_nondet_params.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 0

In [11]:
!mkdir test

A subdirectory or file test already exists.


In [13]:
model_spec = f"stb{boards}_{die}d{sides}"

!mkdir "ShutTheBox/models/$model_spec"
output_model_high = f"ShutTheBox/models/{model_spec}/{model_spec}_high.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_high ShutTheBox/params/stb_params_high.csv

A subdirectory or file ShutTheBox/models/stb12_2d6 already exists.


Model generated successfully at ShutTheBox\models\stb12_2d6\stb12_2d6_high.prism


In [14]:
output_model_low = f"ShutTheBox/models/{model_spec}/{model_spec}_low.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_low ShutTheBox/params/stb_params_low.csv

Model generated successfully at ShutTheBox\models\stb12_2d6\stb12_2d6_low.prism


In [15]:
output_model_nondet = f"ShutTheBox/models/{model_spec}/{model_spec}_nondet.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_nondet ShutTheBox/params/stb_nondet_params.csv

Model generated successfully at ShutTheBox\models\stb12_2d6\stb12_2d6_nondet.prism


In [16]:
output_props = f"ShutTheBox/props/{model_spec}.props"

!python utils/pyprism.py ShutTheBox/stbprops.pyprism $output_props ShutTheBox/params/stb_params_high.csv

Model generated successfully at ShutTheBox\props\stb12_2d6.props


## Step 2: Check properties, run experiments, export data

### Does the game eventually always terminate?

We expect to see a value of approximately 1 here (it won't be exact because CSGs only support methods based on numerical convergence, so it may be slightly under or even over 1!

In [73]:
!prism $output_model_high $output_props -prop 1 | grep -E "(Model checking)|(Result)|(sec)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over ]
Reachable states exploration and model construction done in 6.244 secs.
Time for model construction: 6.424 seconds.
Precomputation took 1.133 seconds.
Time for model checking: 4.586 seconds.
Result: 1.0 (value in the initial state)


### Probability of obtaining each score 

In [74]:
data_location = f"ShutTheBox/data/{model_spec}"

!mkdir "$data_location"

prob_score_high_file = f"{data_location}/{model_spec}_prob_score_high.csv"

!prism $output_model_high $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_high_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

A subdirectory or file ShutTheBox/data/stb12_1d12 already exists.


Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.004629629629629629 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.004918981481481481 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.004340277777777778 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]


In [75]:
prob_score_low_file = f"{data_location}/{model_spec}_prob_score_low.csv"

!prism $output_model_low $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_low_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(sec)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Reachable states exploration and model construction done in 1.887 secs.
Time for model construction: 1.929 seconds.
Precomputation took 0.177 seconds.
Time for model checking: 0.242 seconds.
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Precomputation took 0.147 seconds.
Time for model checking: 0.167 seconds.
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Precomputation took 0.143 seconds.
Time for model checking: 0.16 seconds.
Result: 0.016203703703703703 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Precomputation took 0.119 seconds.
Time for model checking: 0.13 seconds.
Result: 0.015046296296296295 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&sco

In [76]:
prob_score_nondet_min_file = f"{data_location}/{model_spec}_prob_score_nondet_min.csv"

!prism $output_model_nondet $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_nondet_min_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.0011574074074074073 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.004050925925925926 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.002170138888888889 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]

In [77]:
prob_score_nondet_max_file = f"{data_location}/{model_spec}_prob_score_nondet_max.csv"

!prism $output_model_nondet $output_props -prop 3 -const k=1:{max_score} -exportresults {prob_score_nondet_max_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.016203703703703703 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.016203703703703703 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.018518518518518517 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.03038194444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.03211805555555555 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Prop

### Expected number of die rolls

In [78]:
print("High board strategy:")
!prism $output_model_high $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"
print("Low board strategy:")
!prism $output_model_low $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"

High board strategy:
Model checking: <<p1>>R{"no_rolls"}max=? [ F game_over ]
Result: 6.761397545629476 (value in the initial state)
Low board strategy:
Model checking: <<p1>>R{"no_rolls"}max=? [ F game_over ]
Result: 3.7365928822732406 (value in the initial state)


### Probability of obtaining a score given a particular board is covered

In [79]:
for covered_board in range(1, boards+1):
    !prism $output_model_high $output_props -prop {10+covered_board} -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_score_with_covered_{covered_board}.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=4
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=5
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=6
Result: 0.0037615740740740734 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=7
Result: 0.002025462962962963 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F 

Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=2
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=3
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=5
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=6
Result: 0.0037615740740740734 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&score=k ]
Property constants: k=7
Result: 0.002025462962962963 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b2=1&sco

Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=4
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=5
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=6
Result: 0.002604166666666666 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property constants: k=7
Result: 0.0011574074074074073 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b3=1&score=k ]
Property c

Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=5
Result: 0.0023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=6
Result: 0.0011574074074074073 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=7
Result: 0.002025462962962963 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=8
Resu

Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=6
Result: 0.0011574074074074073 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=7
Result: 0.0011574074074074073 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b5=1&score=k ]
Property constants: k=8
Result: 8.68055555555

...

Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b8=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: 

Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b9=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model checking: 

Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b10=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model ch

Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b11=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model ch

Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=5
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=6
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=7
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b12=1&score=k ]
Property constants: k=8
Result: 0.0 (value in the initial state)
Model ch

### Cumulative probability of a score

In [80]:
!prism $output_model_high $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.013888888888888888 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.0162037037037037 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.018518518518518517 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.028067129629629626 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.032407407407407406 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=

In [81]:
!prism $output_model_low $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.013888888888888888 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.030092592592592587 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.04513888888888888 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.06134259259259259 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.09143518518518517 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.11791087962962961 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k 

In [82]:
!prism $output_model_nondet $output_props -prop 5 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.013888888888888888 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.0162037037037037 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.018518518518518517 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.023148148148148147 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.028067129629629626 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.032407407407407406 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=

In [83]:
cum_prob_max_file = "ShutTheBox/data/cum_prob_max.csv"

!prism $output_model_nondet $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_max.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.006944444444444444 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.013888888888888888 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.030092592592592587 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.04513888888888888 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.06134259259259259 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.09143518518518517 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.11791087962962962 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k 

# Optimal Strategy for highest expected score

In [26]:
!prism $output_model_nondet $output_props -prop 7 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>R{"total_boards"}min=? [ F game_over ]
Result: 23.97423197612391 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_min.csv"...


In [27]:
!prism $output_model_nondet $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_max.csv:csv -s | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>R{"total_boards"}max=? [ F game_over ]
Result: 42.91857934707855 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_max.csv"...


In [28]:
!prism $output_model_low $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>R{"total_boards"}max=? [ F game_over ]
Result: 24.30185396277705 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_low.csv"...


In [29]:
!prism $output_model_high $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>R{"total_boards"}max=? [ F game_over ]
Result: 42.808769718128595 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb12_2d6/stb12_2d6_exp_score_high.csv"...
